In [ ]:
# 필요 라이브러리 import
import os
import time
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras import layers
from tensorflow import keras
import numpy as np
from google.colab import drive

In [ ]:
# 상수 선언
d_embed = 100     ## word embedding vector 의 길이 (원소수)
Max_seq_length = 128    ## time step 수 (MSL)
Vocab_size = 51459	# 사전 Vocab 의 총 단어수
Num_POS = 50    ## 총 품사 수
LEARNING_RATE = 0.7e-4
EPOCHS = 20
BATCH_SIZE = 64

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 단어와 그 출현횟수를 dict 타입의 사전에 수집한다.
def build_vocabulary_temp(corpus_path):
    fp = open(corpus_path, "r", encoding="utf-8")
    for line in fp.readlines():
        sentence = line.split()
        if sentence[0] == '<<':
            continue
        for word_pos_pair in sentence:
            w_p = word_pos_pair.split('/')
            nseg = len(w_p)
            ## word/pos 내에 슬래시가 2개 이상 있어 3 조각 이상이 나옴.
            if nseg > 2:    ## 마지막 슬래시를 기준으로 단어와 품사로 구분함.
                word = ''
                for i in range(nseg-1):
                    word = word + w_p[i] + '/'
                word = word[:-1]    # remove the last slash.
            else:
                word = w_p[0]

            pos = w_p[-1]   # the last segment
            if word in Vocab_temporary:
                Vocab_temporary[word] += 1
            else:
                Vocab_temporary[word] = 1
    fp.close()
    return

In [8]:
# 1 차적인 임시사전을 만든다. 사전명: Vocab_temporary
# key: 단어, value:출현횟수
# 수집할 단어들의 대상은 penn-tree-bank 내의 모든 파일에 존재하는 단어들이다!!

Vocab_temporary = {}
build_vocabulary_temp("./drive/MyDrive/all_word_pos_sentences_all.txt")

# 사전 단어들을 출현횟수로 내림차순으로 정렬하여 그 결과를 리스트로 받는다.
sorted_Vocab = sorted(Vocab_temporary.items(), key = lambda kv: kv[1], reverse=True)
Total_n_words = len(sorted_Vocab)
print('파일에서 모은 총 단어수:', Total_n_words)

파일에서 모은 총 단어수: 51457


In [9]:
# 두 개의 특수단어('[PAD]', '[UNK]')를 포함하는 정식 사전(사전명: Vocab) 을 만든다:
#     key: 단어, value: 단어번호
Vocab = {}
Vocab['[PAD]'] = 0  # 특수단어 추가
Vocab['[UNK]'] = 1  # 특수단어 추가
# Vocab_temporary 에 모은 단어들에게는 단어번호를 2 부터 준다.
for i in range(Total_n_words):
    word = sorted_Vocab[i][0]
    freq = sorted_Vocab[i][1]
    Vocab[word] = i + 2

Total_number_words = len(Vocab)
print("최종 사전의 총 단어수:", Total_number_words)

최종 사전의 총 단어수: 51459


In [10]:
## 역-단어사전 i_Vocab  만들기:  Vocab 에서 key와 value 를 바꾼 사전.
#    key: 단어번호,  value: 단어
all_wps = list(Vocab.keys())
i_Vocab = {}
for word in all_wps:
  widx = Vocab[word]
  i_Vocab[widx] = word

In [12]:
## 품사 사전 (사전명: dic_POS)만들기:  key:품사명,  value: 품사번호
#    두 특수단어에게는 품사명/품사번호를  '[PAD]': 0,     '[UNK]': 1 로 준다.
#    나머지는 penn-tree-bank 의 48 개의 품사들에게 품사 번호를 2 부터 부여한다(2~49)
#    결국 총 품사는 총 50 개로 번호는 0 ~ 49 가 된다.
all_pos_list = ['[PAD]', '[UNK]',
                'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS',
                'MD', 'NN', 'NNS', 'NNP', 'NNPS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB',
                'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN',
                'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', '#', '$', '.', ',',
                ':', '(', ')', '\'\'', '\'', '``', '&rsquo', '”']
dic_POS = {}
for i in range(len(all_pos_list)):
    dic_POS[all_pos_list[i]] = i

num_pos = len(dic_POS)
print("총 품사수: ", num_pos)

총 품사수:  50


In [13]:
## 역-품사사전 만들기: dic_POS 에서 key와 value 를 바꾼 사전.
#     key: 품사번호,     value: 품사명
all_pos = list(dic_POS.keys())
i_dic_POS = {}
for a_pos in all_pos:
  pidx = dic_POS[a_pos]
  i_dic_POS[pidx] = a_pos

def build_index_sentences(path_word_pos_sentence_file, path_index_sentence_file):
    fp = open(path_word_pos_sentence_file, "r", encoding="utf-8")
    fp_w = open(path_index_sentence_file, "w", encoding="utf-8")

    for line in fp.readlines():
        sentence = line.split()
        if sentence[0] == '<<':   ## 파일명을 가지는 줄은 무시한다.
            continue

        line_widx = ''  # line for word indices
        line_pidx = ''  # line for pos indices

        for word_pos_pair in sentence:
            w_p = word_pos_pair.split('/')
            nseg = len(w_p)
            if nseg > 2:
                word = ''
                for i in range(nseg-1):
                    word = word + w_p[i] + '/'
                word = word[:-1]    # remove the last slash.
            else:
                word = w_p[0]

            pos = w_p[-1]   # the last segment
            if not(word in Vocab):  # Other scheme: Vocab.get(word) 가 None 이면 없는 것을 맗암.
                widx = 1    # give index of [UNK] since it is missing in Vocab.
            else:
                widx = Vocab[word]

            if not(pos in dic_POS):
                pos_list = pos.split('|')
                pos = pos_list[-1]
                if not (pos in dic_POS):
                    print("exception occurs at dic_POS look_up. w_p=", w_p, " pos=", pos)
                    time.sleep(100)
                else:
                    pidx = dic_POS[pos]
            else:
                pidx = dic_POS[pos]

            if len(line_widx) == 0:
                line_widx = line_widx + str(widx)
            else:
                line_widx = line_widx + '\t' + str(widx)

            if len(line_pidx) == 0:
                line_pidx = line_pidx + str(pidx)
            else:
                line_pidx = line_pidx + '\t' + str(pidx)

        fp_w.write(line_widx + '\n')
        fp_w.write(line_pidx + '\n')
        fp_w.write('\n')    # an empty line after each sentence
    fp_w.close()
    fp.close()

In [17]:

## 단어/품사 문자열을 이용하는 훈련데이타 파일들로 부터  단어번호/품사번호를 이용하는 훈련데이타 파일들을 만든다.
build_index_sentences("./drive/MyDrive/all_word_pos_sentences_train.txt", "./drive/MyDrive/all_index_sentences_train.txt")
build_index_sentences("./drive/MyDrive/all_word_pos_sentences_validation.txt", "./drive/MyDrive/all_index_sentences_validation.txt")
build_index_sentences("./drive/MyDrive/all_word_pos_sentences_test.txt", "./drive/MyDrive/all_index_sentences_test.txt")


##  훈련예제 준비 함수
##   파일경로를 입력으로 받는다. 이 파일은 all_index_sentences_???.txt 이다.
##  출력: 파일내의 모든 문장들에 대한 정보를 이용하여 다음을 준비하여 출력한다.
##       1) list_X: 문장들의 단어 번호 리스트를 원소로 가지는 리스트
##       2) list_Y: 문장들의 정답품사번호 리스트를 원소로 가지는 리스트
##       3) list_leng: 문장들의 길이를 원소로 가지는 리스트

def load_X_and_Y(path_index_file):
    fp= open(path_index_file, "r", encoding="utf-8")
    list_X = []
    list_Y = []
    list_leng = []

    while True:
        # read two lines
        wordline = fp.readline()
        line_leng = len(wordline)

        if line_leng == 0:
            break   # end of file has come.
        if line_leng == 1:
            continue    # empty line used as sentence delimeter

        # The line read just before is a line of word indices.
        # The next line should be the corresponding pos index line.
        posline = fp.readline()
        w_index = wordline.split()
        p_index = posline.split()

        # X : a list of indices of words in a sentence.
        # Y : a list of pos indices of words in the sentence of X.
        X = []
        Y = []

        leng = len(w_index)
        if leng > Max_seq_length:
            leng = Max_seq_length   # truncation is done.

        for i in range(leng):
            X.append(int(w_index[i]))
            Y.append(int(p_index[i]))

        # pads are added after sentence
        if leng < Max_seq_length:
            for i in range(leng, Max_seq_length):
                X.append(0)     # word index of '[PAD]' which is 0 is added.
                Y.append(0)     # pos index of  '[PAD]' which is 0 is added.

        list_X.append(X)
        list_Y.append(Y)
        list_leng.append(leng)

    fp.close()
    return list_X, list_Y, list_leng

In [18]:
x_train, y_train, leng_train = load_X_and_Y("./drive/MyDrive/all_index_sentences_train.txt")
x_train = np.array(x_train, dtype='i')
y_train = np.array(y_train, dtype='i')

x_validation, y_validation, leng_valiation = load_X_and_Y("./drive/MyDrive/all_index_sentences_validation.txt")
x_validation = np.array(x_validation, dtype='i')
y_validation = np.array(y_validation, dtype='i')

x_test, y_test, leng_test = load_X_and_Y("./drive/MyDrive/all_index_sentences_test.txt")
x_test = np.array(x_test, dtype='i')
y_test = np.array(y_test, dtype='i')

In [19]:
# Model 설계

model = tf.keras.models.Sequential()

## 층0:  word-embedding 층
model.add(tf.keras.layers.Embedding(Vocab_size, d_embed, embeddings_initializer='random_normal', \
	input_length=Max_seq_length, mask_zero=True, trainable=True))	# output shape: (bsz, MSL, d_emb)

## 충1: LSTM 층
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True), input_shape=(Max_seq_length, d_embed)))
	# output shape: (batch_sz, MSL, 512)

## 충2: LSTM 층
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True), input_shape=(Max_seq_length, 512)))
	# output shape: (batch_sz, MSL, 256)

## 충3: LSTM 층
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True), input_shape=(Max_seq_length, 256)))
	# output shape: (batch_sz, MSL, 128)

## 층4: NN 층
model.add(tf.keras.layers.Dense(units=Num_POS, activation='softmax', use_bias=True))	# 최종출력층. 각 시간의 각 단어마다 num_POS=50개의 확률이 생성됨.
	# output shape: (batch_sz, MSL, num_POS)

optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=['acc'])
model.fit(x=x_train, y=y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(x_validation, y_validation), shuffle=True, verbose=1)

print("...")

Epoch 1/20
118/844 [===>..........................] - ETA: 47:14 - loss: 3.4278 - acc: 0.1202

KeyboardInterrupt: ignored

In [21]:
##### TEST ####################
pred = model.predict(x=x_test, verbose=1)
pred_label = tf.math.argmax(pred, axis=2)
pred_label = pred_label.numpy()

num_test_sentences = y_test.shape[0]

# 정확도 계산
correct_predictions = 0
total_predictions = 0

for i in range(num_test_sentences):
    for j in range(Max_seq_length):
        if y_test[i][j] == 0:  # [PAD] 토큰은 제외
            continue
        if pred_label[i][j] == y_test[i][j]:
            correct_predictions += 1
        total_predictions += 1

# 테스트 데이터셋의 첫 20 문장에 대해 품사 인식 결과 출력
num_sentences_to_display = 20

for sentence_idx in range(num_sentences_to_display):
    print(f"({sentence_idx + 1}) ", end="")  # 문장 번호 출력
    for word_idx in range(Max_seq_length):
        # PAD 토큰은 무시
        if y_test[sentence_idx][word_idx] == 0:
            break

        # 실제 단어와 품사
        actual_word = i_Vocab[x_test[sentence_idx][word_idx]]
        actual_pos = i_dic_POS[y_test[sentence_idx][word_idx]]

        # 예측된 품사
        predicted_pos = i_dic_POS[pred_label[sentence_idx][word_idx]]

        # 출력 포맷: 올바른 경우 - 단어/품사, 잘못된 경우 - 단어/잘못된품사<정답품사>
        if actual_pos == predicted_pos:
            print(f"{actual_word}/{predicted_pos} ", end="")
        else:
            print(f"{actual_word}/{predicted_pos}<{actual_pos}> ", end="")

    print("\n")  # 문장이 끝나면 줄바꿈



acc = correct_predictions / total_predictions
print("Test Accuracy: ", acc)

print("Program ends.")

240/240 [==============================] - 159s 662ms/step
(1) Arthur/IN<NNP> M./IN<NNP> Goldberg/NN<NNP> said/NN<VBD> he/NN<PRP> extended/NN<VBD> his/NN<PRP$> unsolicited/NN<JJ> tender/NN offer/NN of/NN<IN> $/NN<$> 32/NN<CD> 

(2) a/DT share/NN tender/NN offer/NN ,/NN<,> or/NN<CC> $/NN<$> 154.3/NN<CD> million/NN<CD> ,/NN<,> for/NN<IN> Di/NN<NNP> Giorgio/NN<NNP> Corp./NN<NNP> to/NN<TO> Nov./NN<NNP> 1/NN<CD> ./NN<.> 

(3) DIG/DT<NNP> Acquisition/NN<NNP> Corp./NN<NNP> ,/NN<,> the/NN<DT> New/NN<NNP> Jersey/NN<NNP> investor/NN 's/NN<POS> acquisition/NN vehicle/NN ,/NN<,> said/NN<VBD> that/NN<IN> as/NN<IN> of/NN<IN> the/NN<DT> close/NN of/NN<IN> business/NN yesterday/NN ,/NN<,> 560,839/NN<CD> shares/NN<NNS> had/NN<VBD> been/NN<VBN> tendered/NN<VBN> ./NN<.> 

(4) Including/DT<VBG> the/NN<DT> stake/NN DIG/NN<NNP> already/NN<RB> held/NN<VBD> ,/NN<,> DIG/NN<NNP> holds/NN<VBZ> a/NN<DT> total/NN of/NN<IN> about/NN<RB> 25/NN<CD> %/NN of/NN<IN> Di/NN<NNP> Giorgio/NN<NNP> 's/NN<POS> shares/NN<NNS> o